# Modelling and Analyzing the UAH-DriveSet Using CTHMM

## Introduction

In this notebook, we illustrate how to leverage the **HMMToolkit** for fitting continuous-time hidden Markov models (CTHMM) and performing anomaly detection using the **UAH-DriveSet**. 

In [1]:
using DrWatson
@quickactivate "HMMToolkit"

include(srcdir("HMMToolkit.jl"))
using CSV, DataFrames, Statistics, Distributions, Random, Base.Threads, StatsBase
using .HMMToolkit

The following are some helper functions to prepare accelerometer and GPS data, and to combine the two data sources.  Importantly, to use the **HMMToolkit**, each time series must be identified by a unique **ID**.

In [2]:
## helper functions to prepare accelerometer and gps data, and combine the two data sources
function prepare_accelerometer(folder_name, ID)

    pathname = datadir("UAH-DriveSet", folder_name, "RAW_ACCELEROMETERS.txt")
    df = CSV.read(pathname, DataFrame; delim=' ', header=false)

    df = df[:, 1:8]
    colnames = ["Timestamp", "System_Activated", "X", "Y", "Z", "X_KF", "Y_KF", "Z_KF"]
    rename!(df, colnames)
    df = combine(groupby(df, [:Timestamp]), last)
    sort!(df, [:Timestamp])

    df.time_since = df.Timestamp .- minimum(df.Timestamp)
    df.time_interval = [missing; df.Timestamp[2:end] - df.Timestamp[1:(end-1)]]

    df.ID .= ID
    df.start .= [1; fill(0, nrow(df) - 1)]

    return df

end

function prepare_gps(folder_name, ID)

    pathname = datadir("UAH-DriveSet", folder_name, "RAW_GPS.txt")
    df = CSV.read(pathname, DataFrame; delim=' ', header=false)

    df = df[:, 1:5]
    colnames = ["Timestamp", "Speed", "Latitude", "Longitude", "Altitude"]
    rename!(df, colnames)
    df = combine(groupby(df, [:Timestamp]), last)
    sort!(df, [:Timestamp])

    df.ID .= ID
    df.start .= [1; fill(0, nrow(df) - 1)]

    return df

end

function append_speed(ts_accelerometer, ts_gps)

    idx = [findmin(abs.(ts_accelerometer.Timestamp .- time))[2] for time in ts_gps.Timestamp]
    ts = ts_accelerometer[idx, :]
    ts.Speed .= ts_gps.Speed
    ts = ts[ts.Speed .> 0, :]
    ts.start[1] = 1

    return ts

end

append_speed (generic function with 1 method)

For illustration purposes, we only consider one driver, D2.  Extracting his telematics data:

In [3]:
df_accel = DataFrame()

i = 2  ## only consider driver D2 for illustration purposes
begin
    folder_name = "D" * string(i) * "-Normal1-Secondary"
    df_i = prepare_accelerometer(folder_name, i)
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Normal2-Secondary"
    df_i = prepare_accelerometer(folder_name, (i+5))
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Aggressive-Secondary"
    df_i = prepare_accelerometer(folder_name, (i+10))
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Drowsy-Secondary"
    df_i = prepare_accelerometer(folder_name, (i+15))
    append!(df_accel, df_i)


    folder_name = "D" * string(i) * "-Normal-Motorway"
    df_i = prepare_accelerometer(folder_name, (i+20))
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Aggressive-Motorway"
    df_i = prepare_accelerometer(folder_name, (i+25))
    append!(df_accel, df_i)

    folder_name = "D" * string(i) * "-Drowsy-Motorway"
    df_i = prepare_accelerometer(folder_name, (i+30))
    append!(df_accel, df_i)
end


df_gps = DataFrame()

i = 2  ## only consider driver D2 for illustration purposes
begin
    folder_name = "D" * string(i) * "-Normal1-Secondary"
    df_i = prepare_gps(folder_name, i)
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Normal2-Secondary"
    df_i = prepare_gps(folder_name, (i+5))
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Aggressive-Secondary"
    df_i = prepare_gps(folder_name, (i+10))
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Drowsy-Secondary"
    df_i = prepare_gps(folder_name, (i+15))
    append!(df_gps, df_i)


    folder_name = "D" * string(i) * "-Normal-Motorway"
    df_i = prepare_gps(folder_name, (i+20))
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Aggressive-Motorway"
    df_i = prepare_gps(folder_name, (i+25))
    append!(df_gps, df_i)

    folder_name = "D" * string(i) * "-Drowsy-Motorway"
    df_i = prepare_gps(folder_name, (i+30))
    append!(df_gps, df_i)
end

group_df_accel = groupby(df_accel, :ID)
group_df_gps = groupby(df_gps, :ID)

df_sec = DataFrame()

## combine the two data sources
for i in 1:7

    ts_accelerometer = group_df_accel[i]
    ts_gps = group_df_gps[i]
    ts = append_speed(ts_accelerometer, ts_gps)
    append!(df_sec, ts)

end

group_df = groupby(df_sec, :ID)

for i in 1:length(group_df)
    group_df[i].time_interval = [missing; fill(0.1, nrow(group_df[i])-1)]
end

## Model Fitting

While there are multiple variables in the dataset, we can specify which responses the model should be fitted on.  Here, we only consider lateral acceleration, longitudinal acceleration and speed.

In [4]:
response_list = ["Y_KF", "Z_KF", "Speed"]
## Y_KF for turning
## Z_KF for acceleration and braking

3-element Vector{String}:
 "Y_KF"
 "Z_KF"
 "Speed"

We assume all trips performed by D2 follow the same model, and will train a CTHMM on them.  For model fitting, we need to first specify the number of latent states in the CTHMM, and prepare the training set.

In [5]:
i = 2  ## only consider driver D2 for illustration purposes

num_state = 20

filter_idx = [element in [i, i+5, i+10, i+15, i+20, i+25, i+30] for element in df_sec.ID]  # train with all trips from driver D2
train_set = df_sec[filter_idx, :]

Row,Timestamp,System_Activated,X,Y,Z,X_KF,Y_KF,Z_KF,time_since,time_interval,ID,start,Speed
,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64?,Int64,Int64,Float64
1,15.85,1,-0.022,-0.009,-0.015,-0.006,-0.003,-0.004,15.25,missing,2,1,75.7
2,16.76,1,-0.009,0.023,-0.01,-0.005,0.014,-0.007,16.16,0.1,2,0,75.8
3,17.76,1,0.012,0.032,0.005,-0.004,0.035,0.001,17.16,0.1,2,0,77.5
4,18.76,1,-0.038,0.011,0.003,-0.019,0.004,-0.006,18.16,0.1,2,0,78.9
5,19.76,1,-0.036,-0.046,0.003,-0.026,-0.029,-0.005,19.16,0.1,2,0,79.3
6,20.77,1,0.017,0.0,0.016,-0.014,-0.017,0.009,20.17,0.1,2,0,80.2
7,21.77,1,0.023,0.014,-0.038,0.021,-0.007,-0.017,21.17,0.1,2,0,80.8
8,22.77,1,0.002,0.03,-0.014,0.017,0.013,-0.008,22.17,0.1,2,0,81.6
9,23.77,1,0.005,-0.009,-0.006,0.007,-0.002,-0.006,23.17,0.1,2,0,81.5


Users can specify their own initial guess for parameters in the CTHMM.  Alternatively, the **HMMToolkit** includes functions to provide initial parameter estimates using the Clustered Method of Moments (CMM).

In [6]:
Random.seed!(1234)  ## for reproducibility
block_size = nothing  ## transition matrix not a block matrix; all states can communicate
model_init = CTHMM_cmm_init(train_set[:, response_list], num_state, ["real", "real", "continuous"];
                                block_size = block_size, n_random = 0)
## specify the data type of each response dimension

π_list_init = model_init.π_list_init
Q_mat_init = model_init.Q_mat_init
state_list_init = vcat(vcat([hcat([model_init.params_init[1][j][2] for j in 1:num_state]...) for d in 1:1]...),
                        vcat([hcat([model_init.params_init[2][j][2] for j in 1:num_state]...) for d in 1:1]...),
                        vcat([hcat([model_init.params_init[3][j][1] for j in 1:num_state]...) for d in 1:1]...))
## taking Normal for both acceleration components and Gamma for speed

3×20 Matrix{Main.HMMToolkit.AnyExpert{s, Main.HMMToolkit.NonZI, Distribution{Univariate, Continuous}} where s<:Main.HMMToolkit.ExpertSupport}:
 NormalExpert{Float64}(0.00222115, 0.0156616)    …  NormalExpert{Float64}(0.00186885, 0.0144923)
 NormalExpert{Float64}(-0.000548077, 0.0164689)     NormalExpert{Float64}(0.0050765, 0.0200232)
 GammaExpert{Float64}(68.1276, 0.979934)            GammaExpert{Float64}(314.624, 0.404994)

Now we are ready for model fitting; printing steps for every 10th iteration:

In [7]:
model_fit = CTHMM_learn_EM(train_set, response_list, Q_mat_init, π_list_init, state_list_init;
                            ϵ = 1e-03, max_iter = 1000, Q_max_iter = 1, soft_decode = 1, print_steps = 10, penalty = false, block_size = block_size)

┌ Info: Iteration 10, updating π_list: 10301.019835088207 ->  10301.019838422633, ( + 3.236986151672067e-8 % )
└ @ Main.HMMToolkit c:\Users\ianwe\OneDrive\Documents\GitHub\HMMToolkit\src\CTHMM-learn-EM.jl:93
┌ Info: Iteration 10 sub 1, updating Q: 10301.019838422633 ->  10334.198836120855, ( + 0.3220942995805588 % )
└ @ Main.HMMToolkit c:\Users\ianwe\OneDrive\Documents\GitHub\HMMToolkit\src\CTHMM-learn-EM.jl:122
┌ Info: Iteration 10, updating dim 1: 10334.198836120855 ->  10359.25452299347, ( + 0.24245408153980838 % )
└ @ Main.HMMToolkit c:\Users\ianwe\OneDrive\Documents\GitHub\HMMToolkit\src\CTHMM-learn-EM.jl:147
┌ Info: Iteration 10, updating dim 2: 10359.25452299347 ->  10367.659091545025, ( + 0.0811310170331357 % )
└ @ Main.HMMToolkit c:\Users\ianwe\OneDrive\Documents\GitHub\HMMToolkit\src\CTHMM-learn-EM.jl:147
┌ Info: Iteration 10, updating dim 3: 10367.659091545025 ->  10423.776592651653, ( + 0.541274559773986 % )
└ @ Main.HMMToolkit c:\Users\ianwe\OneDrive\Documents\GitHub\HMMTo

(Q_mat_fit = [-1.3624632739733895 6.631035918692271e-66 … 0.0 1.4511779732805807e-151; 1.251835364969253e-19 -2.428090095995018 … 0.0 7.363498980443832e-120; … ; 0.0 0.0 … -0.6327124564171429 0.10654423098150362; 1.0 1.0 … 1.0 -19.0], π_list_fit = [0.4342769712436041, 0.3149295765331207, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2507934522232751, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.8832039420611694e-257, 7.489003401710251e-196, 1.5388378387964442e-166, 0.0, 0.0], state_list_fit = Main.HMMToolkit.AnyExpert{s, Main.HMMToolkit.NonZI, Distribution{Univariate, Continuous}} where s<:Main.HMMToolkit.ExpertSupport[NormalExpert{Float64}(-0.001343697498061204, 0.03750571290235937) NormalExpert{Float64}(0.0020175988837384962, 0.02445230641990994) … NormalExpert{Float64}(-0.0012920849589351385, 0.024893696782703402) NormalExpert{Float64}(0.0013062351556436102, 0.022772089681077582); NormalExpert{Float64}(0.0028958788521581728, 0.03756690925234109) NormalExpert{Float64}(0.006206242801668494, 0.027287945956686835

And the estimation algorithm converged in 198 iterations.

In [8]:
model_fit.converge, model_fit.iter

(true, 198)

## Unsupervised Anomaly Detection

Having fitted a CTHMM to the 7 trips of D2, we can perform unsupervised anomaly detection to identify outliers and anomalous driving behaviour.  We compute the normal forecast pseudo-residuals of each trip under the fitted model.

In [9]:

ordinary_list, forecast_list = CTHMM_batch_pseudo_residuals(0, train_set, response_list, model_fit.Q_mat_fit, model_fit.π_list_fit, model_fit.state_list_fit;
                                                                keep_last = 1, ZI_list = nothing)
## compute normal pseudo-residuals

(ordinary_list = Array{Float64}[[-0.11765723532982728 -0.2894013703346266 -0.8394934685298964; 0.5802341580925483 -0.3730568361190037 -1.1115390995282501; … ; 0.23809511416674384 -0.6671554363506517 -1.0955698774396438; -0.07497329491995294 0.7913986427249673 -2.04276745970846], [0.8186651087766844 1.3296727467318536 -0.6406868430946567; 1.5713478380761108 2.237885384080189 -0.18860381481185082; … ; 0.16407826843666784 -0.958025179268242 -0.12398606465728054; 0.48758961326768513 0.1278534213624854 -1.4004153625743427], [0.08792883874957305 1.9234912236437434 0.3113859846262809; 0.7728203805244327 1.0495102159724934 -0.14523715981302404; … ; 0.2619798900032585 -0.12246495652130873 -1.291285357138562; 0.0665256510043835 0.7187581858311246 -2.367163338067277], [1.0708740860220844 1.1180953503584263 -0.4499580084865886; 1.2288488565318463 -0.18493861243712956 0.02159381709734081; … ; 0.7588388990261209 -0.0692482762148587 -0.10601870112736445; 0.33009203992894043 0.4464079059195058 -0.3454

We can assess the forecast pseudo-residuals at each data point.

In [10]:
forecast_list[1]

639×3 Matrix{Float64}:
  0.51332    -0.337275   -0.300486
  1.31632    -0.147941   -0.231764
  0.150356   -0.350103    0.134631
 -1.17098    -0.324817    0.269121
 -0.694387    0.053381    0.576391
 -0.293478   -0.647822    0.727504
  0.509234   -0.408216    0.838494
 -0.106671   -0.362112    0.375829
  0.0867517  -0.459294    0.275922
 -0.961603    0.0665221  -0.947623
  ⋮                      
  0.232474   -0.0176326   0.238859
 -0.10122     0.377851    0.384063
 -0.264322    0.17428     0.365298
  0.137076    0.151116    0.335599
  0.441055   -0.408641    0.0900876
  0.160611   -0.385202   -0.275459
  0.160614    0.0345971  -0.588036
  0.207545   -0.689246   -1.11759
 -0.0749733   0.791399   -2.04277

We can also quantify the level of anomalousness of each trip using the anomaly indices (outlier proportions).

In [11]:
CTHMM_anomaly_indices(forecast_list)

7×3 Matrix{Float64}:
 0.00156495  0.00156495  0.0
 0.00294985  0.00147493  0.0
 0.00330579  0.0181818   0.00165289
 0.0126183   0.00473186  0.0
 0.00321888  0.00214592  0.00107296
 0.00586166  0.017585    0.00117233
 0.0140389   0.00215983  0.0

The two aggressive trips are characterized by their highest anomaly indices in longitudinal acceleration (2nd column), while the two drowsy trips are chracterized by their highest anomaly indices in lateral acceleration (1st column).

Alternatively, users can compute the anomaly indices directly from the fitted CTHMM:

In [12]:
CTHMM_batch_anomaly_indices(train_set, response_list, model_fit.Q_mat_fit, model_fit.π_list_fit, model_fit.state_list_fit;
                                keep_last = 1, ZI_list = nothing)

7×3 Matrix{Float64}:
 0.00156495  0.00156495  0.0
 0.00294985  0.00147493  0.0
 0.00330579  0.0181818   0.00165289
 0.0126183   0.00473186  0.0
 0.00321888  0.00214592  0.00107296
 0.00586166  0.017585    0.00117233
 0.0140389   0.00215983  0.0